In [4]:
import pandas as pd
import joblib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

**EDA**

In [26]:
df_2015= pd.read_csv("C:/Users/valen/OneDrive/Documentos/ETL/workshop_3/Workshop_3/data/2015.csv")
df_2016= pd.read_csv("C:/Users/valen/OneDrive/Documentos/ETL/workshop_3/Workshop_3/data/2016.csv")
df_2017= pd.read_csv("C:/Users/valen/OneDrive/Documentos/ETL/workshop_3/Workshop_3/data/2017.csv")
df_2018= pd.read_csv("C:/Users/valen/OneDrive/Documentos/ETL/workshop_3/Workshop_3/data/2018.csv")
df_2019= pd.read_csv("C:/Users/valen/OneDrive/Documentos/ETL/workshop_3/Workshop_3/data/2019.csv")


df_2015.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


Let's check if there are any missing values:

In [16]:
dfs = [df_2015, df_2016, df_2017, df_2018, df_2019]
df_names = ["2015", "2016", "2017", "2018", "2019"]

nulls_info = []

for name, df in zip(df_names, dfs):
    nulls = df.isnull().sum()
    nulls_info.append(pd.DataFrame({
        'Dataset': name,
        'Column': nulls.index,
        'Null Count': nulls.values,
        'Has Nulls': nulls.values > 0
    }))

nulls_info_df = pd.concat(nulls_info, ignore_index=True)

print(nulls_info_df)

   Dataset                         Column  Null Count  Has Nulls
0     2015                        Country           0      False
1     2015                         Region           0      False
2     2015                 Happiness Rank           0      False
3     2015                Happiness Score           0      False
4     2015                 Standard Error           0      False
5     2015       Economy (GDP per Capita)           0      False
6     2015                         Family           0      False
7     2015       Health (Life Expectancy)           0      False
8     2015                        Freedom           0      False
9     2015  Trust (Government Corruption)           0      False
10    2015                     Generosity           0      False
11    2015              Dystopia Residual           0      False
12    2016                        Country           0      False
13    2016                         Region           0      False
14    2016               

I noticed there is one missing value in the 2018's dataset. Let's see the specific row for that NAN:

In [17]:
null_value_row = df_2018[df_2018['Perceptions of corruption'].isnull()]

null_value_row

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
19,20,United Arab Emirates,6.774,2.096,0.776,0.67,0.284,0.186,NaN


United Arab Emirates is the country that has the null value, we are going to replace the value with 0.182 which is the value given by The World Happiness Report in 2018.

In [21]:
df_2018.loc[df_2018['Country or region'] == 'United Arab Emirates', 'Perceptions of corruption'] = df_2018.loc[df_2018['Country or region'] == 'United Arab Emirates', 'Perceptions of corruption'].fillna(0.182)

print(df_2018[df_2018['Country or region'] == 'United Arab Emirates'])

    Overall rank     Country or region  Score  GDP per capita  Social support  \
19            20  United Arab Emirates  6.774           2.096           0.776   

    Healthy life expectancy  Freedom to make life choices  Generosity  \
19                     0.67                         0.284       0.186   

    Perceptions of corruption  
19                      0.182  


                               count       mean        std      min  \
Happiness Rank                 158.0  79.493671  45.754363  1.00000   
Happiness Score                158.0   5.375734   1.145010  2.83900   
Standard Error                 158.0   0.047885   0.017146  0.01848   
Economy (GDP per Capita)       158.0   0.846137   0.403121  0.00000   
Family                         158.0   0.991046   0.272369  0.00000   
Health (Life Expectancy)       158.0   0.630259   0.247078  0.00000   
Freedom                        158.0   0.428615   0.150693  0.00000   
Trust (Government Corruption)  158.0   0.143422   0.120034  0.00000   
Generosity                     158.0   0.237296   0.126685  0.00000   
Dystopia Residual              158.0   2.098977   0.553550  0.32858   
Happiness Rank                 157.0  78.980892  45.466030  1.00000   
Happiness Score                157.0   5.382185   1.141674  2.90500   
Lower Confidence Interval      157.0   5.282395   1.148043  2.73200   
Upper 